# Fine tuning 

In [1]:
from transformers import (Trainer, TrainingArguments, EarlyStoppingCallback,
                          DataCollatorForLanguageModeling, AutoModelForCausalLM, AutoTokenizer)
from datasets import load_dataset
import torch

/home/kevin/MEGA/text_mining/text-mining/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-04 15:09:58.969389: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 15:09:58.988381: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-04 15:09:58.993181: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-04 15:09:59.008538: I tensorflow/core/platform/cpu_feature_guard.cc:210] This 

In [2]:
dataset = load_dataset('mrm8488/CHISTES_spanish_jokes', split='train')
dataset

Dataset({
    features: ['id', 'text', 'keywords', 'funny', 'category'],
    num_rows: 2419
})

In [3]:
dataset.set_format('pandas')
df = dataset.to_pandas()
df.head(10)

,id,text,keywords,funny,category
0,0,"- ¡Rápido, necesitamos sangre!\n- Yo soy 0 pos...",sangre,1,otros
1,1,- ¿Cuál es el mejor portero del mundial? \n- E...,"futbol,porteros",1,otros
2,2,El otro día unas chicas llamarón a mi puerta y...,"dinero,agua",1,otros
3,3,"- Andresito, ¿qué planeta va después de Marte?...",planetas,1,profesiones
4,4,- ¿Por qué Bob Esponja no va al gimnasio? \n- ...,"esponja,gimnasios",1,otros
5,5,Van dos ciegos y le dice uno al otro: \n- Ojal...,ciegos,1,otros
6,6,Noticia de última hora!! \n\nMuere una suegra ...,"canarias,coches,noticias",2,familia
7,7,"– Mamá, mamá, en el colegio dicen que estoy lo...","locos,sillas",1,familia
8,8,"– Mamá, mamá, ¿me haces un bocata de jamón?\n–...","madres,jamón",1,otros
9,9,- Qué pasa si te expulsan de cuatro univerdade...,"universitarios,universidades",1,otros


In [4]:
def jokes_last(df):
    '''
    Agrega <fin> al final de cada chiste si no está presente
    '''
    df['text'] = df['text'].apply(
        lambda x: x.strip() if x.strip().endswith('<fin>') else x.strip() + ' <fin>')
    return df


df = jokes_last(df)
print(df['text'][3])

- Andresito, ¿qué planeta va después de Marte? 
- Miércole, señorita. <fin>


## Preparando el conjunto de datos

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "DeepESP/gpt2-spanish"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model

/home/kevin/MEGA/text_mining/text-mining/env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [6]:
def preprocess_function(max_len):
    def _preprocess_function(examples):
        return tokenizer(
            examples['text'],
            max_length=max_len,
            truncation=True,
            padding='longest',
            return_tensors='pt'
        )
    return _preprocess_function

In [7]:
dataset.reset_format()
tokenized_dataset = dataset.map(preprocess_function(max_len=256), batched=True)
tokenized_dataset = tokenized_dataset.remove_columns([col for col in tokenized_dataset.column_names if col != 'input_ids'])
tokenized_dataset = tokenized_dataset.train_test_split(train_size=0.70)
tokenized_dataset.set_format('torch')
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 1693
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 726
    })
})

In [8]:
# Muestra la primera entrada del dataset tokenizado
print(tokenized_dataset['train'][0])

# lo decodeamos
tokenizer.decode(tokenized_dataset['train'][0]['input_ids'])

{'input_ids': tensor([ 2243, 16195,    21,   268,  2464,   304,  1076, 14752,    21,   314,
         3854,   281,   304, 36008,   293,   324,   426,  2324,    35,   208,
           22,   576,  2776,  4168,   297,  1465,   297, 37595,   296,    40,
          208,    22,  7520,  4199,   307,  2129,  9113,    21, 15558,  3837,
          337, 16064,   313,    23, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 

'Un camarero, de esos un poco pelota, se acerca a un comensal y le dice:\n- ¿Cómo encontró el señor el filete?\n- Pues realmente con mucha dificultad, escondido debajo del puré.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|en

In [9]:
epochs = 10
batch_size = 8
logging_steps = len(tokenized_dataset['train']) // batch_size

# Definimos los parámetros globales de entrenamiento
training_args = TrainingArguments(
    output_dir='./hf-gpt',
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    learning_rate=2e-5,
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    eval_strategy='epoch',  # Cambiado de evaluation_strategy a eval_strategy
    save_strategy='epoch',  # Guardamos al final de cada época
    load_best_model_at_end=True,  # Cargamos el mejor modelo al final
    disable_tqdm=False,
    logging_steps=logging_steps,
    save_total_limit=2,
    # Utilizamos eval_loss como métrica de referencia
    metric_for_best_model="eval_loss",
    greater_is_better=False  # Indica que buscamos minimizar eval_loss
)

# Crear el callback para early stopping, asegurándote de que no se repita
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)

# Definir el entrenador, asegurando que no se añadan múltiples callbacks
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False),
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    callbacks=[early_stopping_callback]  # Solo se añade una vez
)

Descripcion de los argumentos:
- `num_train_epochs`: Número de iteraciones de entrenamiento.
- `learning_rate`: Tasa de aprendizaje para el optimizador, Un valor más alto puede acelerar el entrenamiento, pero un valor demasiado alto puede hacer que el modelo no converja.
- `per_device_train_batch_size`: Tamaño del lote por dispositivo de entrenamiento.
- `per_device_eval_batch_size`: Tamaño del lote por dispositivo de evaluación.
- `weight_decay`: Tasa de decaimiento de los pesos, que ayuda a evitar el sobreajuste al añadir una penalización a los pesos grandes.
- `eval_strategy`: significa que evalua al final de cada epoch.
- `save_strategy`: significa que guarda el modelo al final de cada epoch.
- `load_best_model_at_end`: significa que carga el mejor modelo al final del entrenamiento.
- `logging_steps`: Cada cuántos pasos se imprime el log.

In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.481500,3.176255
2,2.161100,3.216372
3,2.114400,3.227968
4,1.919600,3.261066


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=848, training_loss=2.168072089834033, metrics={'train_runtime': 6211.2886, 'train_samples_per_second': 2.726, 'train_steps_per_second': 0.341, 'total_flos': 3144580551671808.0, 'train_loss': 2.168072089834033, 'epoch': 4.0})

In [33]:
# Guardamos el modelo (solo correr si se vuelve a entrenar el modelo)
#trainer.save_model('trained-gpt2-with-stop')
#tokenizer.save_pretrained('trained-gpt2-with-stop')

('trained-gpt2-with-stop/tokenizer_config.json',
 'trained-gpt2-with-stop/special_tokens_map.json',
 'trained-gpt2-with-stop/vocab.json',
 'trained-gpt2-with-stop/merges.txt',
 'trained-gpt2-with-stop/added_tokens.json',
 'trained-gpt2-with-stop/tokenizer.json')

In [10]:
# Correr para cargar el modelo pre-entrenado
model = AutoModelForCausalLM.from_pretrained('trained-gpt2-with-stop')
tokenizer = AutoTokenizer.from_pretrained('trained-gpt2-with-stop')